In [21]:
%pip install pdf2image pypdf docx2txt --quiet


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
import glob

# To store the documents across all data sources:
all_documents = []

# 1. Load the .pdf document:


In [23]:
loader = PyPDFLoader("data/principles_of_marketing_book.pdf")
pages = loader.load_and_split()

print(pages[0])

page_content='Principles of Mark eting' metadata={'source': 'data/principles_of_marketing_book.pdf', 'page': 0}


In [24]:
# Add extra metadata to each page:
for page in pages:
    page.metadata["description"] = "Principles of Marketing Book"

In [25]:
# Checking that the metadata has been added:
for page in pages[0:2]:
    print(page.metadata)

# Saving the marketing book pages:
all_documents.extend(pages)

{'source': 'data/principles_of_marketing_book.pdf', 'page': 0, 'description': 'Principles of Marketing Book'}
{'source': 'data/principles_of_marketing_book.pdf', 'page': 1, 'description': 'Principles of Marketing Book'}


# 2. Search the data folder for all of the .csv files:


In [26]:
csv_files = glob.glob("data/*.csv")

# Filter to only include the word Marketing in the file name:
csv_files = [f for f in csv_files if "Marketing" in f]

# For each .csv file:
for csv_file in csv_files:
    loader = CSVLoader(file_path=csv_file)
    data = loader.load()
    # Saving the data to the all_documents list:
    all_documents.extend(data)

# 3. Extracting 2 marketing reports from Public Google Cloud Storage.


In [27]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=200, chunk_overlap=0
)

urls = [
    "https://storage.googleapis.com/oreilly-content/NutriFusion%20Foods%20Marketing%20Plan%202022.docx",
    "https://storage.googleapis.com/oreilly-content/NutriFusion%20Foods%20Marketing%20Plan%202023.docx",
]

docs = []
for url in urls:
    loader = Docx2txtLoader(url)
    pages = loader.load()
    chunks = text_splitter.split_documents(pages)
    
    # Adding the metadata to each chunk:
    for chunk in chunks:
        chunk.metadata["source"] = "NutriFusion Foods Marketing Plan - 2022/2023"
    docs.extend(chunks)

# Saving the marketing book pages:
all_documents.extend(docs)